<a href="https://colab.research.google.com/github/william-screen/data-visualization-590/blob/master/labs/Tidy_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 6: Data types and tidy data

## Tidy data

Let's do some tidy exercise first. This is one of the non-tidy dataset assembled by Hadley Wickham (check out [here](https://github.com/tidyverse/tidyr/tree/4c0a8d0fdb9372302fcc57ad995d57a43d9e4337/vignettes) for more datasets, explanation, and R code). 

Let's take a look at this small dataset: https://raw.githubusercontent.com/tidyverse/tidyr/4c0a8d0fdb9372302fcc57ad995d57a43d9e4337/vignettes/pew.csv

In [0]:
import pandas as pd

In [0]:
# pew_df = pd.read_csv('https://raw.githubusercontent.com/tidyverse/tidyr/master/vignettes/pew.csv')
pew_df = pd.read_csv('https://raw.githubusercontent.com/cran/tidyr/master/vignettes/pew.csv')
pew_df

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116
5,Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529
6,Hindu,1,9,7,9,11,34,47,48,54,37
7,Historically Black Prot,228,244,236,238,197,223,131,81,78,339
8,Jehovah's Witness,20,27,24,24,21,30,15,11,6,37
9,Jewish,19,19,25,25,30,95,69,87,151,162


This dataset is about the relationships between income and religion, assembled from a research by the Pew Research Center. You can read more details [here](https://github.com/tidyverse/tidyr/blob/master/vignettes/tidy-data.Rmd#column-headers-are-values-not-variable-names). Is this dataset tidy or not? Why?

Yes, many of the columns are values, not variable names. How should we fix it? 

Pandas provides a convenient function called [melt](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html). You specify the `id_vars` that are variable columns, and `value_vars` that are value columns, and provide the name for the variable as well as the name for the values. 

**Q: so please go ahead and tidy it up! I'd suggest to use the variable name "income" and value name "frequency"**

In [0]:
# TODO: put your code here
pew_tidy_df = pd.melt(pew_df, id_vars=['religion'] ,value_vars=['<$10k', '$10-20k', '$20-30k', '$30-40k', '$40-50k','$50-75k', '$75-100k', '$100-150k', '>150k', 'Don\'t know/refused'], var_name='income', value_name='frequency')

If you were successful, you'll have something like this:

In [0]:
pew_tidy_df.sample(10)

,religion,income,frequency
177,Other Faiths,Don't know/refused,71
88,Other World Religions,$40-50k,2
130,Don’t know/refused,$100-150k,17
15,Other Faiths,<$10k,20
176,Other Christian,Don't know/refused,18
35,Unaffiliated,$10-20k,299
54,Agnostic,$30-40k,81
34,Other World Religions,$10-20k,2
52,Other World Religions,$20-30k,3
21,Catholic,$10-20k,617


## Data types

Let's talk about data types briefly. Understanding data types is not only important for choosing the right visualizations, but also important for efficient computing and storage of data. You may not have thought about how pandas represent data in memory. A Pandas `Dataframe` is essentially a bunch of `Series`, and those `Series` are essentially `numpy` arrays. An array may contain a fixed-length items such as integers or variable length items such as strings. Putting some efforts to think about the correct data type can potentially save a lot of memory as well as time. 

A nice example would be the categorical data type. If you have a variable that only has several possible values, it's essentially a categorical data. Take a look at the `income` variable. 

In [0]:
pew_tidy_df.income.value_counts()

$75-100k              18
$30-40k               18
$40-50k               18
$50-75k               18
$10-20k               18
Don't know/refused    18
$100-150k             18
$20-30k               18
<$10k                 18
>150k                 18
Name: income, dtype: int64

These were the column names in the original messy data. The value can take only one of these income ranges and thus it is a categorical data. What is the data type that pandas use to store this column?

In [0]:
pew_tidy_df.income.dtype

dtype('O')

The `O` means that it is an object data type, which does not have a fixed size like integer or float. The series contains a sort of pointer to the actual text objects. You can actually inspect the amount of memory used by the dataset. 

In [0]:
pew_tidy_df.memory_usage()

Index          80
religion     1440
income       1440
frequency    1440
dtype: int64

In [0]:
pew_tidy_df.memory_usage(deep=True)

Index           80
religion     12780
income       18180
frequency     1440
dtype: int64

What's going on with the `deep=True` option? When you don't specify `deep=True`, the memory usage method just tells you the amount of memory used by the numpy arrays in the pandas dataframe. When you pass `deep=True`, it tells you the total amount of memory by including the memory used by all the text objects. So, the `religion` and `income` columns occupies almost ten times of memory than the `frequency` column, which is simply an array of integers. 

In [0]:
pew_tidy_df.frequency.dtype

dtype('int64')

Is there any way to save up the memory? Note that there are only 10 categories in the income variable. That means we just need 10 numbers to represent the categories! Of course we need to store the names of each category, but that's just one-time cost. The simplest way to convert a column is using `astype` method. 

In [0]:
income_categorical_series = pew_tidy_df.income.astype('category')
# you can do pew_tidy_df.income = pew_tidy_df.income.astype('category')

Now, this series has the `CategoricalDtype` dtype. 

In [0]:
income_categorical_series.dtype

CategoricalDtype(categories=['$10-20k', '$100-150k', '$20-30k', '$30-40k', '$40-50k',
                  '$50-75k', '$75-100k', '<$10k', '>150k',
                  'Don't know/refused'],
                 ordered=False)

How much memory do we use?

In [0]:
income_categorical_series.memory_usage(deep=True)

1590

In [0]:
pew_tidy_df.income.memory_usage(deep=True)

18260

We have reduced the memory usage by almost 10 fold! Not only that, because now the values are just numbers, it will be much faster to match, filter, manipulate. If your dataset is huge, this can save up a lot of space and time. 

If the categories have ordering, you can specify the ordering too. 

In [0]:
from pandas.api.types import CategoricalDtype
income_type = CategoricalDtype(categories=["Don't know/refused", '<$10k', '$10-20k', '$20-30k', '$30-40k', 
                                           '$40-50k', '$50-75k', '$75-100k', '$100-150k', '>150k'], ordered=True)
income_type

CategoricalDtype(categories=['Don't know/refused', '<$10k', '$10-20k', '$20-30k',
                  '$30-40k', '$40-50k', '$50-75k', '$75-100k', '$100-150k',
                  '>150k'],
                 ordered=True)

In [0]:
pew_tidy_df.income.astype(income_type).dtype

CategoricalDtype(categories=['Don't know/refused', '<$10k', '$10-20k', '$20-30k',
                  '$30-40k', '$40-50k', '$50-75k', '$75-100k', '$100-150k',
                  '>150k'],
                 ordered=True)

This data type now allows you to compare and sort based on the ordering. 

**Q: ok, now convert both religion and income columns of `pew_tidy_df` as categorical dtype (in place) and show that `pew_tidy_df` now uses much less memory**

In [0]:
# TODO: put your code here

# Memory usage - BEFORE
print('Memory usage of Religion column: {0}'.format(pew_tidy_df.religion.memory_usage(deep=True)))
print('Memory usage of Income column: {0}'.format(pew_tidy_df.income.memory_usage(deep=True)))

# Convert Religion and Income columns to categorical dtype (in place)
pew_tidy_df.religion = pew_tidy_df.religion.astype('category')
pew_tidy_df.income = pew_tidy_df.income.astype('category')

# Memory usage - AFTER
print('Memory usage of Religion column: {0}'.format(pew_tidy_df.religion.memory_usage(deep=True)))
print('Memory usage of Income column: {0}'.format(pew_tidy_df.income.memory_usage(deep=True)))

Memory usage of Religion column: 12860
Memory usage of Income column: 18260
Memory usage of Religion column: 2199
Memory usage of Income column: 1590


## If you want to know more

- [Jean-Nicholas Hould: Tidy Data in Python](http://www.jeannicholashould.com/tidy-data-in-python.html)
- [Stephen Simmons | Pandas from the Inside](https://www.youtube.com/watch?v=CowlcrtSyME)
- [Data school: How do I make my pandas DataFrame smaller and faster?](https://www.youtube.com/watch?v=wDYDYGyN_cw)

## Visualization

Now we have a well-organized table.
Here, we first generate a bar graph using Altair and edit it on Vega Editor.
Finally, we'll put the figure into a webpage.

First, import Altair. If you didn't install it yet, follow the instruction [here](https://altair-viz.github.io/getting_started/installation.html).

In [0]:
import altair as alt

Basic object of Altair for further process of visualization is "Chart", which takes a dataframe as an input. Let's put our dataframe as an argument for the Chart object.

In [0]:
chart = alt.Chart(pew_tidy_df)

There are multiple ways to visualize it but here we draw a stacked bar chart showing frequencies for each income category with different colors for different religion. 

By using the [example code](https://altair-viz.github.io/gallery/stacked_bar_chart.html) on Altair website, create a stacked bar chart, looks like the following figure.

![Stacked Bar Chart](visualization.png)

In [0]:
# TODO: put your code here

alt.Chart(pew_tidy_df).mark_bar().encode(
    x='religion',
    y='sum(frequency)',
    color='income'
)

Chart({
  data:                     religion              income  frequency
  0                   Agnostic               <$10k         27
  1                    Atheist               <$10k         12
  2                   Buddhist               <$10k         27
  3                   Catholic               <$10k        418
  4         Don’t know/refused               <$10k         15
  5           Evangelical Prot               <$10k        575
  6                      Hindu               <$10k          1
  7    Historically Black Prot               <$10k        228
  8          Jehovah's Witness               <$10k         20
  9                     Jewish               <$10k         19
  10             Mainline Prot               <$10k        289
  11                    Mormon               <$10k         29
  12                    Muslim               <$10k          6
  13                  Orthodox               <$10k         13
  14           Other Christian               <$10k          9
  15              Other Faiths               <$10k         20
  16     Other World Religions               <$10k          5
  17              Unaffiliated               <$10k        217
  18                  Agnostic             $10-20k         34
  19                   Atheist             $10-20k         27
  20                  Buddhist             $10-20k         21
  21                  Catholic             $10-20k        617
  22        Don’t know/refused             $10-20k         14
  23          Evangelical Prot             $10-20k        869
  24                     Hindu             $10-20k          9
  25   Historically Black Prot             $10-20k        244
  26         Jehovah's Witness             $10-20k         27
  27                    Jewish             $10-20k         19
  28             Mainline Prot             $10-20k        495
  29                    Mormon             $10-20k         40
  ..                       ...                 ...        ...
  150                    Hindu               >150k         54
  151  Historically Black Prot               >150k         78
  152        Jehovah's Witness               >150k          6
  153                   Jewish               >150k        151
  154            Mainline Prot               >150k        634
  155                   Mormon               >150k         42
  156                   Muslim               >150k          6
  157                 Orthodox               >150k         46
  158          Other Christian               >150k         12
  159             Other Faiths               >150k         41
  160    Other World Religions               >150k          4
  161             Unaffiliated               >150k        258
  162                 Agnostic  Don't know/refused         96
  163                  Atheist  Don't know/refused         76
  164                 Buddhist  Don't know/refused         54
  165                 Catholic  Don't know/refused       1489
  166       Don’t know/refused  Don't know/refused        116
  167         Evangelical Prot  Don't know/refused       1529
  168                    Hindu  Don't know/refused         37
  169  Historically Black Prot  Don't know/refused        339
  170        Jehovah's Witness  Don't know/refused         37
  171                   Jewish  Don't know/refused        162
  172            Mainline Prot  Don't know/refused       1328
  173                   Mormon  Don't know/refused         69
  174                   Muslim  Don't know/refused         22
  175                 Orthodox  Don't know/refused         73
  176          Other Christian  Don't know/refused         18
  177             Other Faiths  Don't know/refused         71
  178    Other World Religions  Don't know/refused          8
  179             Unaffiliated  Don't know/refused        597
  
  [180 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      shorthand: 'income'
    }),
    x: X({
      shorthand: 'r

Looks good, except the orientation - it seems better if we can turn it about 90 degrees so the vertical axis becomes income while the horizontal axis becomes frequency.

Let's move to Vega Editor to change the size of figure and other small features. 

If you click ![menu button](menu1.png) at the rigt top side of the figure, 
you can see the menus including **Save as SVG**, **Save as PNG**, ..., **Open in Vega Editor**.

In this editor, you can conver the orientation of this figure by changing values of x-axis and y-axis each other.

Does it look like the following figure now?

![new stacked bar chart](visualization2.png)

The bars still look uneven. Can we sort the bars so that the longest bar is at the top? Your figure should look like this:
![sorted stacked bar chart](visualization3.png)

In [0]:
# TODO: put your code here
alt.Chart(pew_tidy_df).mark_bar().encode(
    x='sum(frequency)',
    y=alt.Y(
      'religion:N',
      sort=alt.EncodingSortField(
        field="frequency",  # The field to use for the sort
        op="sum",  # The operation to run on the field prior to sorting
        order="descending"  # The order to sort in
      )
    ),
    color='income'
)

Chart({
  data:                     religion              income  frequency
  0                   Agnostic               <$10k         27
  1                    Atheist               <$10k         12
  2                   Buddhist               <$10k         27
  3                   Catholic               <$10k        418
  4         Don’t know/refused               <$10k         15
  5           Evangelical Prot               <$10k        575
  6                      Hindu               <$10k          1
  7    Historically Black Prot               <$10k        228
  8          Jehovah's Witness               <$10k         20
  9                     Jewish               <$10k         19
  10             Mainline Prot               <$10k        289
  11                    Mormon               <$10k         29
  12                    Muslim               <$10k          6
  13                  Orthodox               <$10k         13
  14           Other Christian               <$10k          9
  15              Other Faiths               <$10k         20
  16     Other World Religions               <$10k          5
  17              Unaffiliated               <$10k        217
  18                  Agnostic             $10-20k         34
  19                   Atheist             $10-20k         27
  20                  Buddhist             $10-20k         21
  21                  Catholic             $10-20k        617
  22        Don’t know/refused             $10-20k         14
  23          Evangelical Prot             $10-20k        869
  24                     Hindu             $10-20k          9
  25   Historically Black Prot             $10-20k        244
  26         Jehovah's Witness             $10-20k         27
  27                    Jewish             $10-20k         19
  28             Mainline Prot             $10-20k        495
  29                    Mormon             $10-20k         40
  ..                       ...                 ...        ...
  150                    Hindu               >150k         54
  151  Historically Black Prot               >150k         78
  152        Jehovah's Witness               >150k          6
  153                   Jewish               >150k        151
  154            Mainline Prot               >150k        634
  155                   Mormon               >150k         42
  156                   Muslim               >150k          6
  157                 Orthodox               >150k         46
  158          Other Christian               >150k         12
  159             Other Faiths               >150k         41
  160    Other World Religions               >150k          4
  161             Unaffiliated               >150k        258
  162                 Agnostic  Don't know/refused         96
  163                  Atheist  Don't know/refused         76
  164                 Buddhist  Don't know/refused         54
  165                 Catholic  Don't know/refused       1489
  166       Don’t know/refused  Don't know/refused        116
  167         Evangelical Prot  Don't know/refused       1529
  168                    Hindu  Don't know/refused         37
  169  Historically Black Prot  Don't know/refused        339
  170        Jehovah's Witness  Don't know/refused         37
  171                   Jewish  Don't know/refused        162
  172            Mainline Prot  Don't know/refused       1328
  173                   Mormon  Don't know/refused         69
  174                   Muslim  Don't know/refused         22
  175                 Orthodox  Don't know/refused         73
  176          Other Christian  Don't know/refused         18
  177             Other Faiths  Don't know/refused         71
  178    Other World Religions  Don't know/refused          8
  179             Unaffiliated  Don't know/refused        597
  
  [180 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      shorthand: 'income'
    }),
    x: X({
      shorthand: 's

Now, put this figure into the html file to publish it on the web.

Open the `result.html` file in this folder, and put your code in Vega Editor in the right format with the right variable, 
following [this example](https://vega.github.io/vega-lite/tutorials/getting_started.html#embed).

In [0]:
# TODO: Submit the result.html including your figure.
alt.Chart(pew_tidy_df).mark_bar().encode(
    x='sum(frequency)',
    y=alt.Y(
      'religion:N',
      sort=alt.EncodingSortField(
        field="frequency",  # The field to use for the sort
        op="sum",  # The operation to run on the field prior to sorting
        order="descending"  # The order to sort in
      )
    ),
    color='income'
).save('result.html')